# Рекомендация тарифов
## Описание проекта
Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Требуется модель с максимально большим значением accuracy и довести долю правильных ответов по крайней мере до 0.75.
## План проекта:
1. Открыть файл с данными и изучить его.
2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.
3. Исследовать качество разных моделей, меняя гиперпараметры. Кратко написать выводы исследования.
4. Проверить качество модели на тестовой выборке.
5. Дополнительное задание: проверить модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми работали раньше.

## Описание данных
* Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 1. Откроем и изучим данные

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from joblib import dump
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import random

try:
    meg = pd.read_csv('users_behavior.csv')
except:
    meg = pd.read_csv('/datasets/users_behavior.csv')
    
meg

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [2]:
meg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
meg.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [4]:
# посмотрим на соотношение классов в выборке:
meg.is_ultra.value_counts( )

0    2229
1     985
Name: is_ultra, dtype: int64

Итак, у нас 5 столбцов и 3214 строк без NaN значений. Все переменные имеют нужные типы данных. При этом стоит обратить внимание, что тарифы smart и ultra занимают по 70% и 30%, соответственно 

## 2. Разделим данные на обучающую, валидационную и тестовую выборки.
**Определим признаки:**
* Нам нужно построить модель для задачи классификации, которая выберет подходящий тариф. Следовательной, целевой признак - столбец **is_ultra**;

Для построения модели нужно разделить данные на выборки. **Отделим 20% на валидационную выборку и еще 20% на тестовую, остальное на обучающую**

In [5]:
features = meg.drop(['is_ultra'], axis=1)
target = meg['is_ultra']
#Выделяем валидационную выборку
features_train_test, features_valid, target_train_test, target_valid = train_test_split(features, target, test_size=0.2, random_state=12345, stratify=target)

#Выделяем тестовую выборку
features_train, features_test, target_train, target_test = train_test_split(features_train_test, target_train_test, test_size=0.2, random_state=12345, stratify=target_train_test)


## 3. Исследуем качество разных моделей, меняя гиперпараметры. Кратко опишем выводы.

Будем сравнивать следующие модели:
* дерево решений,
* случайный лес,
* логистическую регрессию.

### Decision tree:

In [6]:
# тренируем на обучающей выборке
# предсказываем на валидационной и с помощью цикла выявим max_depth 

dt_best_model = None
dt_best_result = 0
dt_best_max_depth = 0

for depth in range(1, 6):
    dt_model = DecisionTreeClassifier(max_depth=depth,random_state=12345)
    dt_model.fit(features_train,target_train)
    dt_predictions = dt_model.predict(features_valid)
    dt_result = accuracy_score(target_valid, dt_predictions)
    print("max_depth =", depth, ": ", end='==> ')
    print('accuracy_score =',dt_result)
    if dt_result > dt_best_result:
        dt_best_model = dt_model
        dt_best_result = dt_result
        dt_best_max_depth = depth
        
print('-------------------------------------------------')
print("Accuracy наилучшей модели на валидационной выборке:", dt_best_result)
print('Max_depth: ', dt_best_max_depth)

max_depth = 1 : ==> accuracy_score = 0.7480559875583204
max_depth = 2 : ==> accuracy_score = 0.7853810264385692
max_depth = 3 : ==> accuracy_score = 0.7993779160186625
max_depth = 4 : ==> accuracy_score = 0.7931570762052877
max_depth = 5 : ==> accuracy_score = 0.8040435458786936
-------------------------------------------------
Accuracy наилучшей модели на валидационной выборке: 0.8040435458786936
Max_depth:  5


### Random forest

In [7]:
# тренируем на обучающей выборке
# считаем качество модели на валидационной выборке 
# и с помощью цикла выявим оптимальное количество деревьев 

rf_best_model = None
rf_best_result = 0
rf_best_est = 0

for est in range(1, 11):
    rf_model = RandomForestClassifier(random_state=12345, n_estimators=est)
    rf_model.fit(features_train,target_train) 
    rf_result = rf_model.score(features_valid,target_valid)
    print('n_estimators =',est,'==> accuracy_score = ',rf_result)
    if rf_result > rf_best_result:
        rf_best_model = rf_model 
        rf_best_result = rf_result
        rf_best_est = est
print('-------------------------------------------------')
print("Accuracy наилучшей модели на валидационной выборке:", rf_best_result)
print('Количество деревьев = ', rf_best_est)

n_estimators = 1 ==> accuracy_score =  0.7433903576982893
n_estimators = 2 ==> accuracy_score =  0.7636080870917574
n_estimators = 3 ==> accuracy_score =  0.7620528771384136
n_estimators = 4 ==> accuracy_score =  0.7978227060653188
n_estimators = 5 ==> accuracy_score =  0.7869362363919129
n_estimators = 6 ==> accuracy_score =  0.8055987558320373
n_estimators = 7 ==> accuracy_score =  0.7962674961119751
n_estimators = 8 ==> accuracy_score =  0.8055987558320373
n_estimators = 9 ==> accuracy_score =  0.7978227060653188
n_estimators = 10 ==> accuracy_score =  0.8055987558320373
-------------------------------------------------
Accuracy наилучшей модели на валидационной выборке: 0.8055987558320373
Количество деревьев =  6


In [8]:
# тренируем на обучающей выборке
# считаем качество модели на валидационной выборке 
# и с помощью цикла выявим оптимальное количество деревьев 

rf_best_model = None
rf_best_result = 0
rf_best_est = 0
rf_max_depth = 0

for est in range(100, 1001, 100):
    for depth in range(1,6):
        rf_model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        rf_model.fit(features_train,target_train) 
        rf_result = rf_model.score(features_valid,target_valid)
        print('n_estimators =',est,'==> accuracy_score = ',rf_result)
        if rf_result > rf_best_result:
            rf_best_model = rf_model 
            rf_best_result = rf_result
            rf_best_est = est
            rf_max_depth = depth
print('-------------------------------------------------')
print("Accuracy наилучшей модели на валидационной выборке:", rf_best_result)
print('Количество деревьев = ', rf_best_est)
print('Глубина деревьев: ', rf_max_depth)

n_estimators = 100 ==> accuracy_score =  0.7744945567651633
n_estimators = 100 ==> accuracy_score =  0.7853810264385692
n_estimators = 100 ==> accuracy_score =  0.7993779160186625
n_estimators = 100 ==> accuracy_score =  0.80248833592535
n_estimators = 100 ==> accuracy_score =  0.8102643856920684
n_estimators = 200 ==> accuracy_score =  0.7698289269051322
n_estimators = 200 ==> accuracy_score =  0.7838258164852255
n_estimators = 200 ==> accuracy_score =  0.8009331259720062
n_estimators = 200 ==> accuracy_score =  0.80248833592535
n_estimators = 200 ==> accuracy_score =  0.8102643856920684
n_estimators = 300 ==> accuracy_score =  0.7744945567651633
n_estimators = 300 ==> accuracy_score =  0.7853810264385692
n_estimators = 300 ==> accuracy_score =  0.8009331259720062
n_estimators = 300 ==> accuracy_score =  0.8009331259720062
n_estimators = 300 ==> accuracy_score =  0.8118195956454122
n_estimators = 400 ==> accuracy_score =  0.7511664074650077
n_estimators = 400 ==> accuracy_score =  0.7

### Logistic  regression

In [9]:
lr_model = LogisticRegression(random_state=12345, solver = 'lbfgs', max_iter = 1000)
lr_model.fit(features_train, target_train)
lr_result = lr_model.score(features_valid, target_valid)
dump(lr_model, 'model_9_1.joblib')

lr_result

0.7107309486780715

**Вывод:** Видим, что самой "качественной" моделью оказалась Random forest с accuracy = 81.2%, количеством деревьев = 300 и max_depth = 5

## 4. Проверяем качество модели на тестовой выборке.

### тестируем наилучшую Decision tree модель на тестовой выборке:

In [10]:
dt_test_predictions = dt_best_model.predict(features_test)
print('accuracy_score =',accuracy_score(target_test, dt_test_predictions))

accuracy_score = 0.8077669902912621


### тестируем наилучшую Random forest модель на тестовой выборке:

In [11]:
rf_test_predictions = rf_best_model.predict(features_test)
print('accuracy_score =',accuracy_score(target_test, rf_test_predictions))

accuracy_score = 0.8155339805825242


### тестируем наилучшую Logistic regression модель на тестовой выборке:

In [12]:
lr_test_predictions = lr_model.predict(features_test)
print('accuracy_score =',accuracy_score(target_test, lr_test_predictions))

accuracy_score = 0.7067961165048544


**Вывод:** Наилучшая модель на тестовой выборке оказалась Random forest

## 5. Дополнительное задание: проверить модели на вменяемость.
Проверим дополнительно модели через метрики качества: `recall` и `precision`

In [13]:
print('recall_score of DecisionTreeClassifier =',recall_score(target_test, dt_test_predictions))
print('recall_score of RandomForestClassifier =',recall_score(target_test, rf_test_predictions))
print('recall_score of LogisticRegression =',recall_score(target_test, lr_test_predictions))

recall_score of DecisionTreeClassifier = 0.5
recall_score of RandomForestClassifier = 0.5253164556962026
recall_score of LogisticRegression = 0.06962025316455696


#### Мы видим, что самой "полной" моделью считается также Random forest. То есть из всех правильных ответов он обнаружил 52,5%

In [14]:
print('precision_score of DecisionTreeClassifier =',precision_score(target_test, dt_test_predictions))
print('precision_score of RandomForestClassifier =',precision_score(target_test, rf_test_predictions))
print('precision_score of LogisticRegression =',precision_score(target_test, lr_test_predictions))

precision_score of DecisionTreeClassifier = 0.797979797979798
precision_score of RandomForestClassifier = 0.8058252427184466
precision_score of LogisticRegression = 0.7333333333333333


#### Самой точной однако оказалась также Random forest: из тех ответов, которые модель пометила маркером, действительно правильны из них около 80.6%

In [15]:
# здесь я услышал на zoom собрании, мол, что если зарандомить в fit. Возможно я не совсем понял данный момент

# dt_best_model1 = None
# dt_best_result1 = 0
# dt_best_max_depth1 = 0

# for depth in range(1, 6):
#     dt_model1 = DecisionTreeClassifier(max_depth=depth,random_state=12345)
#     dt_model1.fit(features_train,target_train.sample(len(target_train)))
#     dt_predictions1 = dt_model1.predict(features_valid)
#     dt_result1 = accuracy_score(target_valid, dt_predictions1)
#     print("max_depth =", depth, ": ", end='==> ')
#     print('accuracy_score =',dt_result1)
#     if dt_result1 > dt_best_result1:
#         dt_best_model1 = dt_model1
#         dt_best_result1 = dt_result1
#         dt_best_max_depth1 = depth
        
# print('-------------------------------------------------')
# print("Accuracy наилучшей модели на валидационной выборке:", dt_best_result1)
# print('Max_depth: ', dt_best_max_depth1)

In [16]:
one = [1] * len(target_train)
ones = pd.Series(one)
ones

0       1
1       1
2       1
3       1
4       1
       ..
2051    1
2052    1
2053    1
2054    1
2055    1
Length: 2056, dtype: int64

In [17]:
# проверим вменяемость модели на DecisionTreeClassifier

dt_best_model1 = None
dt_best_result1 = 0
dt_best_max_depth1 = 0

for depth in range(1, 6):
    dt_model1 = DecisionTreeClassifier(max_depth=depth,random_state=12345)
    dt_model1.fit(features_train, ones)
    dt_predictions1 = dt_model1.predict(features_valid)
    dt_result1 = accuracy_score(target_valid, dt_predictions1)
    print("max_depth =", depth, ": ", end='==> ')
    print('accuracy_score =',dt_result1)
    if dt_result1 > dt_best_result1:
        dt_best_model1 = dt_model1
        dt_best_result1 = dt_result1
        dt_best_max_depth1 = depth
        
print('-------------------------------------------------')
print("Accuracy наилучшей модели на валидационной выборке:", dt_best_result1)
print('Max_depth: ', dt_best_max_depth1)

max_depth = 1 : ==> accuracy_score = 0.30637636080870917
max_depth = 2 : ==> accuracy_score = 0.30637636080870917
max_depth = 3 : ==> accuracy_score = 0.30637636080870917
max_depth = 4 : ==> accuracy_score = 0.30637636080870917
max_depth = 5 : ==> accuracy_score = 0.30637636080870917
-------------------------------------------------
Accuracy наилучшей модели на валидационной выборке: 0.30637636080870917
Max_depth:  1


## Вывод:

Итак, мы знаем, что в целом:
* Самое высокое качество (accuracy) у случайного леса;
* На втором месте — логистическая регрессия. Модель несложная, а значит, переобучение ей не грозит.
* Самое низкое качество предсказания у дерева решений. Если глубина меньше четырёх, оно недообучается, когда больше — переобучается.

На практике мы действительно получили, что самое высокое качество имеет модель Случайного леса. Однако на втором месте оказалась модель Дерево решений и на последнем - Логистическая регрессия